# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symred.utils import unit_n_sphere_cartesian_coords
from symred.symplectic_form import PauliwordOp, QuantumState, array_to_QuantumState
from functools import cached_property
from typing import Union

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.0000000000 |0> ->  0.0000000000+1.0000000000j |1>
 1.0000000000 |1> ->  0.0000000000-1.0000000000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.0000000000 |00>

After application of the Hadamard gate:

 0.5000000000+0.0000000000j |00> +
 0.5000000000+0.0000000000j |01> +
 0.5000000000+0.0000000000j |10> +
 0.5000000000-0.0000000000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

0.5000000000+0.0000000000j ZZ +
0.5000000000+0.0000000000j ZX +
0.5000000000+0.0000000000j XZ +
0.5000000000-0.0000000000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 4 # number of qubits
M = 2 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.2430739504+0.0909935595j |0101> +
 0.3324373365+0.9067086876j |0010>

 0.3302040854+0.1350497740j |0101> +
 0.7956724053+0.4895224652j |0001>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.5382674365+0.3311589050j |0001> +
 0.2248917918+0.6133827913j |0010> +
 0.3878190279+0.1529169101j |0101>

Norm: (1.0000000000000002+0j)


In [8]:
op = random_herm_op(N, 15)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

-0.2752208861+0.0000000000j XZZX +
1.2070865378+0.0000000000j XYII +
-40.9043454528+0.0000000000j IYIX +
80.8677045434+0.0000000000j IZII +
-0.4272038189+0.0000000000j IIZZ +
0.6282262417+0.0000000000j ZXIZ +
-0.0907285754+0.0000000000j IIIZ +
-1.0000000000+0.0000000000j ZIZI +
11.0218858381+0.0000000000j ZIYI +
1.5917832361+0.0000000000j XXYY +
-2.3408030449+0.0000000000j XZII +
0.0123658759+0.0000000000j XIXI +
-0.0244472803+0.0000000000j ZIII +
0.8284410179+0.0000000000j IIXX +
-3.6334451727+0.0000000000j ZXIX

Expectation value <psi|op|psi> = (53.241045117643914-3.552713678800501e-15j)


In [9]:
op_psi = op * psi
print(op_psi)

-0.9034501904+12.8291366247j |0000> +
 43.1984830467+27.0244086192j |0001> +
 18.9274701149+50.6819793330j |0010> +
-3.6499956451+5.9327222357j |0011> +
 11.5900730308-23.2207248939j |0100> +
-31.8966244003-12.6515369116j |0101> +
 0.4625681153+0.5120258063j |0110> +
 22.5874568687-7.1532472319j |0111> +
-0.1453614568-0.0835568318j |1000> +
-1.0753941108-1.2433089008j |1001> +
 0.0908964362-1.1923977315j |1010> +
 0.0685510665+0.1729108253j |1011> +
 0.1067358965+0.0420859275j |1100> +
 0.8660494893+1.9840561896j |1101> +
 0.1163989721+0.7985970478j |1110> +
 0.0047957220+0.0018909515j |1111>


In [10]:
psi.conjugate * op_psi

(53.241045117643914+3.552713678800501e-15j)

It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [11]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

 0.4082482905 |0000000000> +
 0.5773502692 |0011111010> +
 0.7071067812 |0111110100>
1.0
